https://github.com/chanzuckerberg/single-cell-curation/issues/513 <br>
https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/4.0.0/schema.md#column_colors

In [1]:
#import cxg_upload
import matplotlib.colors as mcolors
import numpy as np
import os
import pandas as pd
import random
import scanpy as sc
import subprocess
from datetime import datetime

In [2]:
def validate(file):
    validate_process = subprocess.run(['cellxgene-schema', 'validate', file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in validate_process.stdout.decode('utf-8').split('\n'):
        print(line)
    for line in validate_process.stderr.decode('utf-8').split('\n'):
        print(line)
        if 'is_valid=' in line:
            valid = line.split('=')[-1]
            return valid

In [3]:
def save_and_test(adata, expected):
    now = datetime.now() 
    dt_string = now.strftime('%m/%d %H:%M')
    adata.uns['title'] += (' - ' + dt_string)

    file = 'test.h5ad'
    adata.write(filename=file)
    test_adata = sc.read_h5ad(file)
    for k,v in test_adata.uns.items():
        if k.endswith('_colors'):
            print('--')
            print(k)
            print(v)
            obs_field = k.replace('_colors','')
            if obs_field in test_adata.obs.columns:
                uniq_vals = str(len(test_adata.obs[obs_field].unique()))
                val_type = str(test_adata.obs[obs_field].dtype)
                print(obs_field + ':' + uniq_vals + ' values of type:' + val_type)
            else:
                print(obs_field + ' not in obs')
            print('--')
    print('------------------')

    valid = validate(file)
    print('------------------')
    if expected != valid:
        print('\033[1m\033[91mERROR\033[0m')
    else:
        print('\033[1m\033[92mPASSED\033[0m')
        #if expected == 'True':
            #cxg_upload.upload(file)
    os.remove(file)

In [4]:
adata = sc.read_h5ad('../valid.h5ad')

In [5]:
df = pd.DataFrame(adata.obs.dtypes).reset_index()
df['unique len'] = df['index'].apply(lambda x: len(adata.obs[x].unique()))
df.set_index('index', inplace=True)
df

,0,unique len
index,,
BICCN_cluster_id,float64,61
QC,category,2000
BICCN_cluster_label,category,61
BICCN_subclass_label,category,17
BICCN_class_label,category,3
cluster_color,category,61
size,float64,55
temp_class_label,category,17
BICCN_ontology_term_id,category,17


In [6]:
def create_color_list(count):
    # Generating a random number in between 0 and 2^24
    colors = random.sample(range(0, 2**24),count)

    # Converting that number from base-10
    # (decimal) to base-16 (hexadecimal)
    hex_colors = [hex(c).replace('0x','#') for c in colors]

    return hex_colors

**Test valid cases**

In [7]:
#untouched, no colors defined
save_and_test(adata, 'True')

------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:01.652165 with status is_valid=True
------------------
PASSED


In [8]:
#colors matches length of obs - named & hex
adata.uns['title'] = '513 colors matches length of obs'

adata.uns['temp_class_label_colors'] = np.array(create_color_list(17))
adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 3))
save_and_test(adata, 'True')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_53465/1194014541.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 3))


--
BICCN_class_label_colors
['turquoise' 'orchid' 'bisque']
BICCN_class_label:3 values of type:category
--
--
temp_class_label_colors
['#e91b64' '#c90c01' '#514bd0' '#7bdb5' '#da05c' '#b52d0a' '#e9dde5'
 '#c6e0aa' '#56d49b' '#e1c165' '#f78ec7' '#ce064e' '#7d6bae' '#f52cb'
 '#1da543' '#ae2240' '#db787f']
temp_class_label:17 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[temp_class_label_colors] must be either all hex colors or all CSS4 named colors. Found: ['#1da543' '#514bd0' '#56d49b' '#7bdb5' '#7d6bae' '#ae2240' '#b52d0a'
 '#c6e0aa' '#c90c01' '#ce064e' '#da05c' '#db787f' '#e1c165' '#e91b64'
 '#e9dde5' '#f52cb' '#f78ec7']
Validation complete in 0:00:00.596298 with status is_valid=False
------------------
ERROR


In [9]:
#colors exceeds length of obs - named & hex
adata.uns['title'] = '513 colors exceeds length of obs'

adata.uns['temp_class_label_colors'] = np.array(create_color_list(40))
adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 12))
save_and_test(adata, 'True')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_53465/3889217428.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 12))


--
BICCN_class_label_colors
['yellow' 'mistyrose' 'gray' 'darkgreen' 'antiquewhite' 'palegreen'
 'whitesmoke' 'darkslategrey' 'paleturquoise' 'cyan' 'ghostwhite'
 'indianred']
BICCN_class_label:3 values of type:category
--
--
temp_class_label_colors
['#d3498a' '#56dd6c' '#eb7513' '#4bdc45' '#62aed1' '#2f3792' '#70c7f7'
 '#35bd66' '#c8b763' '#18307d' '#fda82a' '#b252f6' '#ee9567' '#4ce7e6'
 '#bad0f6' '#f46a6c' '#77f849' '#e5675c' '#c26785' '#49df11' '#c6aa61'
 '#a6d215' '#c2a3a7' '#d7203d' '#5cd1db' '#e20879' '#d4db8' '#4b8353'
 '#760942' '#e97fd2' '#359284' '#99c53d' '#4417d6' '#e60a42' '#759de6'
 '#14ad8a' '#12d6d2' '#a72da7' '#776949' '#710145']
temp_class_label:17 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be either all hex colors or all CSS4 named colors. Found: ['antiquewhite' 'cyan' 'darkgreen' 'darkslategrey' 'ghostwhite' 'gray'
 'indianred' 'mistyrose' '

In [10]:
adata.uns = {'title': 'temporary title'}
#colors for non-schema *_ontology_term_id - hex
adata.uns['title'] = '513 colors for non-schema *_ontology_term_id'

adata.uns['BICCN_ontology_term_id_colors'] = np.array(create_color_list(17))
save_and_test(adata, 'True')

--
BICCN_ontology_term_id_colors
['#c2f515' '#9535f3' '#7b90b5' '#8e0abb' '#932f39' '#842102' '#d14d3a'
 '#be0a38' '#41763e' '#c1dea5' '#61072' '#db8fd1' '#7d3624' '#c3347f'
 '#624095' '#9fc916' '#9b09f6']
BICCN_ontology_term_id:17 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_ontology_term_id_colors] must be either all hex colors or all CSS4 named colors. Found: ['#41763e' '#61072' '#624095' '#7b90b5' '#7d3624' '#842102' '#8e0abb'
 '#932f39' '#9535f3' '#9b09f6' '#9fc916' '#be0a38' '#c1dea5' '#c2f515'
 '#c3347f' '#d14d3a' '#db8fd1']
Validation complete in 0:00:00.524219 with status is_valid=False
------------------
ERROR


In [11]:
#colors for non-schema_ontology_term_id - named
adata.uns['title'] = '513 colors for non-schema_ontology_term_id'

adata.uns['BICCN_ontology_term_id_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 17))
save_and_test(adata, 'True')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_53465/565956949.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_ontology_term_id_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 17))


--
BICCN_ontology_term_id_colors
['gray' 'lightslategray' 'springgreen' 'skyblue' 'plum' 'fuchsia'
 'aliceblue' 'slateblue' 'dodgerblue' 'yellow' 'moccasin' 'teal' 'navy'
 'beige' 'darkslategrey' 'powderblue' 'mediumblue']
BICCN_ontology_term_id:17 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_ontology_term_id_colors] must be either all hex colors or all CSS4 named colors. Found: ['aliceblue' 'beige' 'darkslategrey' 'dodgerblue' 'fuchsia' 'gray'
 'lightslategray' 'mediumblue' 'moccasin' 'navy' 'plum' 'powderblue'
 'skyblue' 'slateblue' 'springgreen' 'teal' 'yellow']
Validation complete in 0:00:00.523417 with status is_valid=False
------------------
ERROR


In [12]:
adata.uns = {'title': 'temporary title'}
#colors is for *_ontology_term_id = hex * named
adata.uns['title'] = '513 colors is for *_ontology_term_id'

adata.uns['sex_ontology_term_id_colors'] = np.array(create_color_list(2))
adata.uns['assay_ontology_term_id_colors'] = random.sample(mcolors.CSS4_COLORS.keys(), 1)
save_and_test(adata, 'True')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_53465/3666815808.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['assay_ontology_term_id_colors'] = random.sample(mcolors.CSS4_COLORS.keys(), 1)


--
assay_ontology_term_id_colors
['lavenderblush']
assay_ontology_term_id:1 values of type:category
--
--
sex_ontology_term_id_colors
['#f07304' '#e18eac']
sex_ontology_term_id:2 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
Validation complete in 0:00:01.629299 with status is_valid=True
------------------
PASSED


In [13]:
adata.uns = {'title': 'temporary title'}
#duplicate colors in a list - hex & named
adata.uns['title'] = '513 duplicate colors in a list'

adata.uns['BICCN_class_label_colors'] = random.sample(mcolors.CSS4_COLORS.keys(), 2)
adata.uns['BICCN_class_label_colors'] += adata.uns['BICCN_class_label_colors']
adata.uns['BICCN_class_label_colors'] = np.array(adata.uns['BICCN_class_label_colors'])

adata.uns['cell_type_ontology_term_id_colors'] = create_color_list(3)
adata.uns['cell_type_ontology_term_id_colors'] += adata.uns['cell_type_ontology_term_id_colors']
adata.uns['cell_type_ontology_term_id_colors'] = np.array(adata.uns['cell_type_ontology_term_id_colors'])
save_and_test(adata, 'True')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_53465/83740981.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_class_label_colors'] = random.sample(mcolors.CSS4_COLORS.keys(), 2)


--
BICCN_class_label_colors
['dodgerblue' 'oldlace' 'dodgerblue' 'oldlace']
BICCN_class_label:3 values of type:category
--
--
cell_type_ontology_term_id_colors
['#c6b71b' '#db098a' '#a60ac8' '#c6b71b' '#db098a' '#a60ac8']
cell_type_ontology_term_id:6 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Annotated categorical field BICCN_class_label must have at least 3 color options in uns[BICCN_class_label_colors]. Found: ['dodgerblue' 'oldlace']
ERROR: Annotated categorical field cell_type_ontology_term_id must have at least 6 color options in uns[cell_type_ontology_term_id_colors]. Found: ['#a60ac8' '#c6b71b' '#db098a']
Validation complete in 0:00:00.641408 with status is_valid=False
------------------
ERROR


**Test invalid cases**

In [14]:
adata.uns = {'title': 'temporary title'}
#invalid test for length of colors array is less than obs counterpart - named & hex
adata.uns['temp_class_label_colors'] = np.array(create_color_list(2))
adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 1))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/2281613395.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 1))


--
BICCN_class_label_colors
['powderblue']
BICCN_class_label:3 values of type:category
--
--
temp_class_label_colors
['#670d17' '#550496']
temp_class_label:17 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: ['powderblue']
ERROR: Colors in uns[temp_class_label_colors] must be strings. Found: ['#670d17' '#550496']
Validation complete in 0:00:00.514451 with status is_valid=False
------------------
PASSED


In [15]:
adata.uns = {'title': 'temporary title'}
#invalid test for colors array that is empty np array
adata.uns['BICCN_class_label_colors'] = np.array([])
save_and_test(adata, 'False')

--
BICCN_class_label_colors
[]
BICCN_class_label:3 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: []
Validation complete in 0:00:00.512304 with status is_valid=False
------------------
PASSED


In [16]:
adata.uns = {'title': 'temporary title'}
#invalid test for colors array that is None or np.nan
adata.uns['BICCN_class_label_colors'] = np.array([np.nan, np.nan, np.nan])
save_and_test(adata, 'False')

--
BICCN_class_label_colors
[nan nan nan]
BICCN_class_label:3 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: [nan nan nan]
Validation complete in 0:00:00.528823 with status is_valid=False
------------------
PASSED


In [17]:
adata.uns = {'title': 'temporary title'}
#invalid test for colors array that is empty strings
adata.uns['BICCN_class_label_colors'] = np.array(['','',''])
save_and_test(adata, 'False')

--
BICCN_class_label_colors
['' '' '']
BICCN_class_label:3 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: ['' '' '']
Validation complete in 0:00:00.601835 with status is_valid=False
------------------
PASSED


In [18]:
adata.uns = {'title': 'temporary title'}
#invalid test for colors as string or dictionary
adata.uns['BICCN_class_label_colors'] = ','.join(create_color_list(17))
adata.uns['sex_ontology_term_id_colors'] = ','.join(random.sample(mcolors.CSS4_COLORS.keys(), 2))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/1445415300.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['sex_ontology_term_id_colors'] = ','.join(random.sample(mcolors.CSS4_COLORS.keys(), 2))


--
BICCN_class_label_colors
#68952e,#2f1f73,#cc5c4e,#7ee86b,#39f382,#5a4a25,#e1c908,#cf4ce0,#4225c5,#ac2690,#881d6f,#ce3921,#418e1b,#fc02ae,#ab2e4,#6bd49b,#3cda6b
BICCN_class_label:3 values of type:category
--
--
sex_ontology_term_id_colors
lime,teal
sex_ontology_term_id:2 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors field uns['BICCN_class_label_colors'] must be of 'numpy.ndarray' type, it is <class 'str'>
ERROR: Colors field uns['sex_ontology_term_id_colors'] must be of 'numpy.ndarray' type, it is <class 'str'>
Validation complete in 0:00:00.541761 with status is_valid=False
------------------
PASSED


In [19]:
adata.uns = {'title': 'temporary title'}
#colors for schema fields - named & hex
adata.uns['cell_type_colors'] = np.array(create_color_list(6))
adata.uns['sex_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 2))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/1083785059.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['sex_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 2))


--
cell_type_colors
['#d2520f' '#59900b' '#149ee6' '#330c9a' '#3f6020' '#5c566']
cell_type not in obs
--
--
sex_colors
['cornflowerblue' 'dimgrey']
sex not in obs
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors field uns[cell_type_colors] does not have a corresponding categorical field in obs
ERROR: Colors field uns[sex_colors] does not have a corresponding categorical field in obs
Validation complete in 0:00:00.533900 with status is_valid=False
------------------
PASSED


In [20]:
adata.uns = {'title': 'temporary title'}
#colors does not have obs counterpart
adata.uns['author_cell_type_colors'] = np.array(create_color_list(17))
save_and_test(adata, 'False')

--
author_cell_type_colors
['#4ed5f3' '#382692' '#703dde' '#492ab3' '#5ced6a' '#e7a77c' '#32753c'
 '#cc57c8' '#5b33a0' '#528643' '#d7cbfc' '#32dbc' '#255c41' '#6ba963'
 '#82d2fa' '#77cd2c' '#5783b5']
author_cell_type not in obs
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors field uns[author_cell_type_colors] does not have a corresponding categorical field in obs
Validation complete in 0:00:00.516170 with status is_valid=False
------------------
PASSED


In [21]:
adata.uns = {'title': 'temporary title'}
#colors is mixed named & hex
adata.uns['BICCN_ontology_term_id_colors'] = \
    np.array(create_color_list(20) + random.sample(mcolors.CSS4_COLORS.keys(), 20))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/386928218.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  np.array(create_color_list(20) + random.sample(mcolors.CSS4_COLORS.keys(), 20))


--
BICCN_ontology_term_id_colors
['#f5b0f6' '#e4c5a1' '#7172e9' '#db066d' '#6c19a' '#94535f' '#c0e567'
 '#f27bb7' '#2bed14' '#6b9cc2' '#9add3d' '#3e1dfe' '#4a2347' '#72d8a0'
 '#60920f' '#391d74' '#8292a' '#90b26d' '#4b8895' '#ba6896'
 'darkslategrey' 'lightseagreen' 'saddlebrown' 'lightgoldenrodyellow'
 'mediumblue' 'steelblue' 'gold' 'darkorange' 'lightblue' 'dodgerblue'
 'darkmagenta' 'olive' 'brown' 'lightgray' 'greenyellow' 'lavender'
 'gainsboro' 'mediumslateblue' 'navy' 'slategray']
BICCN_ontology_term_id:17 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_ontology_term_id_colors] must be strings. Found: ['#f5b0f6' '#e4c5a1' '#7172e9' '#db066d' '#6c19a' '#94535f' '#c0e567'
 '#f27bb7' '#2bed14' '#6b9cc2' '#9add3d' '#3e1dfe' '#4a2347' '#72d8a0'
 '#60920f' '#391d74' '#8292a' '#90b26d' '#4b8895' '#ba6896'
 'darkslategrey' 'lightseagreen' 'saddlebrown' 'lightgoldenrodyellow'
 'mediumblue' 's

In [22]:
adata.uns = {'title': 'temporary title'}
#colors is non-named or hex
adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.BASE_COLORS.keys(), 3))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/2724578257.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.BASE_COLORS.keys(), 3))


--
BICCN_class_label_colors
['y' 'c' 'w']
BICCN_class_label:3 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: ['y' 'c' 'w']
Validation complete in 0:00:00.522807 with status is_valid=False
------------------
PASSED


In [23]:
#colors is non-named or hex
adata.uns['BICCN_class_label_colors'] = np.array(random.sample(list(mcolors.BASE_COLORS.values()), 3))
save_and_test(adata, 'False')

--
BICCN_class_label_colors
[[0.   0.5  0.  ]
 [0.   0.75 0.75]
 [0.75 0.   0.75]]
BICCN_class_label:3 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: [[0.   0.5  0.  ]
 [0.   0.75 0.75]
 [0.75 0.   0.75]]
Validation complete in 0:00:00.518634 with status is_valid=False
------------------
PASSED


In [24]:
#colors is non-named or hex
adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.XKCD_COLORS.keys(), 3))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/3751173854.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.XKCD_COLORS.keys(), 3))


--
BICCN_class_label_colors
['xkcd:dark sage' 'xkcd:pistachio' 'xkcd:peacock blue']
BICCN_class_label:3 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: ['xkcd:dark sage' 'xkcd:pistachio' 'xkcd:peacock blue']
Validation complete in 0:00:00.537274 with status is_valid=False
------------------
PASSED


In [25]:
#colors is non-named or hex
adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.TABLEAU_COLORS.keys(), 3))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/3878135180.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['BICCN_class_label_colors'] = np.array(random.sample(mcolors.TABLEAU_COLORS.keys(), 3))


--
BICCN_class_label_colors
['tab:orange' 'tab:cyan' 'tab:brown']
BICCN_class_label:3 values of type:category
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors in uns[BICCN_class_label_colors] must be strings. Found: ['tab:orange' 'tab:cyan' 'tab:brown']
Validation complete in 0:00:00.523151 with status is_valid=False
------------------
PASSED


In [26]:
adata.uns = {'title': 'temporary title'}
#colors counterpart in obs is boolean
adata.uns['is_primary_data_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 1))
save_and_test(adata, 'False')

/var/folders/63/6nnd9y1x6y769sbxd81cy8380000gn/T/ipykernel_6839/3542767481.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  adata.uns['is_primary_data_colors'] = np.array(random.sample(mcolors.CSS4_COLORS.keys(), 1))


--
is_primary_data_colors
['rebeccapurple']
is_primary_data:1 values of type:bool
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors field uns[is_primary_data_colors] does not have a corresponding categorical field in obs
Validation complete in 0:00:00.535543 with status is_valid=False
------------------
PASSED


In [27]:
adata.uns = {'title': 'temporary title'}
#colors counterpart in obs is float
adata.uns['size_colors'] = np.array(create_color_list(17))
save_and_test(adata, 'False')

--
size_colors
['#92a9ab' '#44b4c' '#a48b58' '#8fa2' '#244f84' '#a18f46' '#fa9622'
 '#483586' '#fbbe8' '#65597' '#d75471' '#aa2177' '#b0cadd' '#aaf7c8'
 '#23602c' '#7ce633' '#a23b93']
size:55 values of type:float64
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors field uns[size_colors] does not have a corresponding categorical field in obs
Validation complete in 0:00:00.587618 with status is_valid=False
------------------
PASSED


In [28]:
adata.uns = {'title': 'temporary title'}
adata.obs['BICCN_cluster_id'] = adata.obs['BICCN_cluster_id'].map(int)
#colors counterpart in obs is int
adata.uns['BICCN_cluster_id_colors'] = np.array(create_color_list(61))
save_and_test(adata, 'False')

--
BICCN_cluster_id_colors
['#22edce' '#2b0ce2' '#210182' '#2467d1' '#278e44' '#ef12d0' '#605157'
 '#e4d019' '#6c2bc6' '#5afa6a' '#ab20bd' '#cabe1a' '#9e9f76' '#78011c'
 '#20b591' '#e0f43a' '#a3b8cf' '#f0ee36' '#a69935' '#c1c121' '#b4f2cc'
 '#6bee47' '#9ab08c' '#149608' '#f10b18' '#cd8a39' '#6150cf' '#98841e'
 '#4bff25' '#3f7e0' '#2f602' '#829037' '#a4243b' '#823b63' '#472f46'
 '#aaa883' '#46326b' '#b989c0' '#fb7e27' '#bb3399' '#43c67' '#3d8ef2'
 '#1d032' '#df06db' '#c7235' '#18f3c5' '#e3f918' '#844597' '#f78fb1'
 '#e93cf8' '#1d38a0' '#5ffd7b' '#1aa4e0' '#c1ffbb' '#c179ea' '#6d4839'
 '#6a1a72' '#c11eda' '#1d559' '#9a59ee' '#169f5c']
BICCN_cluster_id:61 values of type:int64
--
------------------
Loading dependencies
Loading validator modules

Starting validation...
ERROR: Colors field uns[BICCN_cluster_id_colors] does not have a corresponding categorical field in obs
Validation complete in 0:00:00.518460 with status is_valid=False
------------------
PASSED


In [29]:
#not np.array - couldn't test as array seems to be converted upon writing AnnData